In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

## Q2. Using Beautiful Soup to scrape data from the webpage

In [22]:
#Import the wikipedia page containing postal codes for Toronto
import requests
wiki_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [23]:
#Parse data into a BeautifulSoup object
soup=BeautifulSoup(wiki_data,'html.parser')

In [24]:
#Create lists to store the scraped data
postalCode = []
borough = []
neighborhood = []

In [25]:
#Get the table for Toronto postal code by scraping data from the wikipedia page
table=soup.find_all('table')[0]
for rows in table.find_all('tr'):
    cells=rows.find_all('td')
    if len(cells)>0:
        postalCode.append(cells[0].get_text())
        borough.append(cells[1].get_text())
        neighborhood.append(cells[2].get_text())
number_of_rows=len(postalCode)
for rows in range(number_of_rows):
    postalCode[rows]=postalCode[rows].rstrip('\n')
    borough[rows]=borough[rows].rstrip('\n')
    neighborhood[rows]=neighborhood[rows].rstrip('\n')

In [26]:
#Put the data into a pandas dataframe
df=pd.DataFrame({'Postal Code':postalCode,'Borough':borough,'Neighborhood':neighborhood})
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Successfully scraped the data and converted to a pandas dataframe. 

## Q3. Cleaning the data:


In [27]:
#Remove all Boroughs that are not assigned
df_toronto=df[df.Borough!='Not assigned'].reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [28]:
#For cases where a Neighborhood does not have an assigned value, the neighborhood will be same as the borough
df_toronto.groupby(["Postal Code", "Borough"],as_index=False)
df_toronto['Neighborhood']=df_toronto['Neighborhood'].replace('Not assigned',df_toronto['Borough'])
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [29]:
df_toronto.shape

(103, 3)

## Cleaned the data to get the required dataframe

## Q4. Get the location for the postal codes and add to dataframe

In [30]:
#Get the file containing geo data and convert to pandas dataframe
geo_url='http://cocl.us/Geospatial_data'
geo_data=pd.read_csv(geo_url)

In [34]:
#Add latitude and longitude data to dataframe
toronto_data=pd.merge(df_toronto,geo_data,on="Postal Code")
toronto_data.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Q4. Exploring and clustering the neighborhoods in Toronto 
(Only boroughs with the word Toronto in it will be considered)

In [35]:
#Import the requried libraries 
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [36]:
#get latitute and longitude of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto City are 43.6534817, -79.3839347.


In [38]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the boroughs which have the word Toronto in them. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [42]:
toronto_borough_data=toronto_data[toronto_data["Borough"].str.contains("Toronto")].reset_index(drop=True)
toronto_borough_data.size

195

In [43]:
toronto_borough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [44]:
#These are the differen boroughs in our new dataset
toronto_borough_data["Borough"].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

In [46]:
# create map of Toronto using latitude and longitude values
#The markers display the boroughs that have the word Toronto in them
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(toronto_borough_data['Latitude'], toronto_borough_data['Longitude'], toronto_borough_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we will use FourSquare API to explore neighborhoods and segment them using K-means clustering

## Define FourSquare Credentials and Version


In [47]:
CLIENT_ID = 'OC4QCCJI4ITBBFWO0FGGHWTDTJXNO34KBX44MS3VD5PEG1IT' # your Foursquare ID
CLIENT_SECRET = 'BXZUJOPBRVYL4BXB2VV0QFI1LE3BTHCC2PYUACPKGTU4CYSG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OC4QCCJI4ITBBFWO0FGGHWTDTJXNO34KBX44MS3VD5PEG1IT
CLIENT_SECRET:BXZUJOPBRVYL4BXB2VV0QFI1LE3BTHCC2PYUACPKGTU4CYSG


## Let's explore the first neighborhood in the dataset

In [48]:
toronto_borough_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_borough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_borough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_borough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


Now, let's find the top 100 venues within a 500m radius in this neighborhood


In [49]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OC4QCCJI4ITBBFWO0FGGHWTDTJXNO34KBX44MS3VD5PEG1IT&client_secret=BXZUJOPBRVYL4BXB2VV0QFI1LE3BTHCC2PYUACPKGTU4CYSG&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [61]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [72]:
results = requests.get(url).json()['response']['groups'][0]['items']
from pandas.io.json import json_normalize
nearby_venues = json_normalize(results) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


## We have only explored one neighborhood so far. Let's repeat the process for all neighborhoods for the dataset below

In [75]:
toronto_borough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


## Let's create a function that does the above procedure. We can repeatedly call the function for all our neighborhoods

In [77]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [79]:
#Add venues for all enighborhoods to a new dataset
toronto_venues = getNearbyVenues(names=toronto_borough_data['Neighborhood'],
                                   latitudes=toronto_borough_data['Latitude'],
                                   longitudes=toronto_borough_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [80]:
#Lets' check the size of the dataframe
print(toronto_venues.shape)
toronto_venues.head()

(1634, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [81]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,66,66,66,66,66,66
Christie,16,16,16,16,16,16
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Lets' see how many unique categories are there in Venue Categories

In [82]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Let's analyze each neighborhood and create a dataframe which will contain the 10 most frequent venues in each neighborhood. This will be then be used for K-means clustering

In [88]:
#Let's assign a dummy value for each category in Venue Category

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
loc=toronto_onehot.columns.get_loc("Neighborhood")
fixed_columns = [toronto_onehot.columns[loc]] + list(toronto_onehot.columns[0:loc]) +list(toronto_onehot.columns[loc+1:])
toronto_onehot = toronto_onehot[fixed_columns] 
toronto_onehot

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's examine the size of the new dataframe

In [89]:
toronto_onehot.shape

(1634, 234)

In [90]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.017544,...,0.000000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.015152,0.000000,0.000000,0.015152,0.000000,0.015152
5,Christie,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.000,0.000,0.000,0.013889,0.000000,0.00,0.000000,...,0.013889,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's examine the size of the resulting dataframe

In [92]:
toronto_grouped.shape

(39, 234)

Finally, let's create the dataframe that contains top 10 venues for each neighborhood

In [93]:
#Function that sorts venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Beer Bar,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Pet Store,Furniture / Home Store,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Auto Workshop,Brewery,Burrito Place,Spa,Farmers Market,Fast Food Restaurant,Restaurant,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Rental Car Location,Bar,Coffee Shop,Boat or Ferry,Plane,Boutique
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Department Store,Salad Place


## Cluster the neighborhoods

In [121]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 0, 0, 3, 0, 0, 3, 3], dtype=int32)

Let's create a dataframe which contains the clusters as well as top 10 venues for each neighborhood

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ValueError: cannot insert Cluster Labels, already exists

In [123]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Ice Cream Shop,French Restaurant,Health Food Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Diner,Yoga Studio,Café,Bar,Beer Bar,Italian Restaurant,Burrito Place,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Café,Bubble Tea Shop,Hotel,Bookstore,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Seafood Restaurant,Lingerie Store,Farmers Market
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Health Food Store,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Café,Beer Bar,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Seafood Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Department Store,Salad Place
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Diner,Coffee Shop,Candy Store,Italian Restaurant,Baby Store,Restaurant,Nightclub
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Gym,Thai Restaurant,Deli / Bodega,Salad Place,Bookstore
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,3,Bakery,Pharmacy,Music Venue,Café,Middle Eastern Restaurant,Supermarket,Recording Studio,Bar,Bank,Pet Store


In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Let's examine all the clusters
By doing so, we can analyze the distinguishing features for each cluster

### Cluster 1

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Ice Cream Shop,French Restaurant,Health Food Store
1,Downtown Toronto,0,Coffee Shop,College Cafeteria,Diner,Yoga Studio,Café,Bar,Beer Bar,Italian Restaurant,Burrito Place,Sandwich Place
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Café,Bubble Tea Shop,Hotel,Bookstore,Fast Food Restaurant
3,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Seafood Restaurant,Lingerie Store,Farmers Market
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Café,Beer Bar,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Seafood Restaurant
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Department Store,Salad Place
8,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Gym,Thai Restaurant,Deli / Bodega,Salad Place,Bookstore
10,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,Baseball Stadium
11,West Toronto,0,Bar,Men's Store,Asian Restaurant,Café,Vietnamese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Record Shop,Cocktail Bar
13,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Salad Place,Japanese Restaurant,American Restaurant,Sushi Restaurant


### Cluster 2

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1,Park,Bus Line,Swim School,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
21,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
33,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 3

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Garden,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 4

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Pub,Health Food Store,Trail,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
7,Downtown Toronto,3,Grocery Store,Café,Park,Diner,Coffee Shop,Candy Store,Italian Restaurant,Baby Store,Restaurant,Nightclub
9,West Toronto,3,Bakery,Pharmacy,Music Venue,Café,Middle Eastern Restaurant,Supermarket,Recording Studio,Bar,Bank,Pet Store
12,East Toronto,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Furniture / Home Store,Restaurant,Ice Cream Shop,Yoga Studio,Spa,Bookstore,Juice Bar
14,West Toronto,3,Café,Breakfast Spot,Coffee Shop,Pet Store,Furniture / Home Store,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection
15,East Toronto,3,Sandwich Place,Park,Pizza Place,Sushi Restaurant,Pub,Brewery,Fast Food Restaurant,Fish & Chips Shop,Restaurant,Italian Restaurant
17,East Toronto,3,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Gastropub,Yoga Studio,Food,Pet Store,Park
20,Central Toronto,3,Gym / Fitness Center,Hotel,Pizza Place,Dance Studio,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gas Station
22,West Toronto,3,Mexican Restaurant,Café,Thai Restaurant,Flea Market,Discount Store,Diner,Fried Chicken Joint,Bar,Speakeasy,Bakery
24,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Liquor Store,Donut Shop,Indian Restaurant,Pub,BBQ Joint,History Museum,Pizza Place


### Cluster 5

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,4,Playground,Summer Camp,Tennis Court,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## Observations
1. Most neighborhoods fall into Cluster 1. This could be the business disticy because it has a lot of coffee shops, restaurants, pubs
2. The second biggest cluster is 4 which seems like a mix of residential/ business because in addition to restaurants, it also has gyms , movie theatres, pet stores spas
3. The other clusters seem to be residential areas, probably occupied by immigrants from North Africa, Middle-East and Eastern Europe because of the abundance of restaurants serving those cuisines